In [1]:
import re
from pandas import Series, DataFrame
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import seaborn as sns
from scipy import stats
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Read files
df_raw = pd.read_csv(r'C:\Users\Abraham\MSBA\Text\HW1\Train_rev1.csv')
df_train = df_raw.sample(2500)

# Part A1

In [ ]:
df_train = df_train[['Id','FullDescription','SalaryNormalized']]
df_train['FullDescription'] = df_train['FullDescription'].map(str.lower)

In [ ]:
pos_dt = {}

def pos_count(desc):
    tokens = re.findall('\w+', desc)
    tokenlist = nltk.pos_tag(tokens)
    
    for t in tokenlist:
        pos = t[1]
        if pos in pos_dt:
            pos_dt[pos] += 1
        else:
            pos_dt[pos] = 1
    
    return desc

df_train['FullDescription'].map(pos_count)

pos_dt


In [ ]:
stop_words = set(stopwords.words('english'))

pos_nostop_dt = {}

def pos_nostop_count(desc):
    tokens = re.findall('\w+', desc)
    filtered_tokens = [w for w in tokens if not w in stop_words]
    
    tokenlist = nltk.pos_tag(filtered_tokens)
    
    for t in tokenlist:
        pos = t[1]
        if pos in pos_nostop_dt:
            pos_nostop_dt[pos] += 1
        else:
            pos_nostop_dt[pos] = 1
    
    return desc

df_train['FullDescription'].map(pos_nostop_count)

pos_nostop_dt


In [ ]:
Series(pos_dt).sort_values(ascending=False)[0:5].plot(kind='bar')

In [ ]:
Series(pos_nostop_dt).sort_values(ascending=False)[0:5].plot(kind='bar')

# Part A2

In [ ]:
a = df_train['FullDescription'].str.cat(sep = ' ') #pos = a.decode('utf-8')
s = re.findall('\w+', a)
s = Series(s)

df_words = DataFrame(s.value_counts())
df_words = df_words.reset_index()
df_words.columns = ['Word','Counts']





# topcount = df_words.iloc[0]['Counts']
# zipf_list = []

# for f in range(1,len(df_words)):
#     zipf_list.append(topcount/f)

# Series(zipf_list)
    
# df_words['Zipf'] = Series(zipf_list)

# df_words['Counts'][0:50].plot(kind='bar')
# df_words['Zipf'][0:50].plot(kind='line')



In [ ]:
df_words['Rank'] = df_words['Counts'].rank(ascending=False)
df_words['LogCount'] = log(df_words['Counts']/df_words.iloc[len(df_words)-1]['Counts'])
df_words['LogRank'] = log(df_words['Rank'])

slope, intercept, r_value, p_value, std_err = stats.linregress(df_words['LogCount'],df_words['LogRank'])
ax=sns.regplot(x='LogCount', y='LogRank', data=df_words,color='b',line_kws={'label':"y={0:.1f}x+{1:.1f}".format(slope,intercept)})
ax.legend()
plt.show()


In [ ]:
s = re.findall('\w+', a)
s = [w for w in s if not w in stop_words]

lemmatizer = WordNetLemmatizer()
s = Series(s).map(lemmatizer.lemmatize)

In [ ]:
s.value_counts()[0:10].plot(kind='bar')

# Part B1

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [ ]:
df_train = df_raw.dropna().sample(2500)
cutoff = df_train['SalaryNormalized'].quantile(0.75)

df_train['HighSalary'] = 0
df_train['HighSalary'][df_train['SalaryNormalized']>cutoff] = 1

df_train.head()

In [ ]:
df_nbtrain = pd.get_dummies(df_train[['ContractType','ContractTime','Category']])

#Cities where avg rent is >= 20% of avg income
high_cities = ['London', 'Central London','The City','Brighton','Edinburgh','Bristol','Southampton','Portsmouth','Exeter'\
                 ,'Cardiff', 'Manchester', 'Birmingham', 'Leeds', 'Aberdeen', 'Glasgow', 'Newcastle', 'Sheffield', 'Liverpool', 'Hull']

a = df_train['LocationNormalized']
b = []

for i in range(len(a)):
    if a.iloc[i] in high_cities: 
        b.append(1)
    else: b.append(0)

df_nbtrain['ExpensiveCity'] = b
df_nbtrain['HighSalary'] = df_train['HighSalary']
df_nbtrain = df_nbtrain.drop(['ContractType_part_time','ContractTime_contract'], axis=1)
df_nbtrain.head()


In [ ]:
from sklearn import naive_bayes
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from patsy import dmatrices
from sklearn import metrics

In [ ]:
dummy_column_names = list(df_nbtrain.columns.values)[:-1]

formula = 'HighSalary ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))

nbtrain, nbtest = train_test_split(df_nbtrain, test_size=0.2)
    
#Training Sets
Ytrain, xtrain = dmatrices(formula, nbtrain, return_type='dataframe')
ytrain = Ytrain['HighSalary'].values

#Testing Set
Ytest, xtest = dmatrices(formula, nbtest, return_type='dataframe')
ytest = Ytest['HighSalary'].values

model = naive_bayes.MultinomialNB()
model.fit(xtrain, ytrain)

prediction_train = model.predict(xtrain)
print ("Training")
print (metrics.accuracy_score(ytrain, prediction_train))

print (" ")
print ("Test")
prediction_test = model.predict(xtest)
print (metrics.accuracy_score(ytest, prediction_test))

In [ ]:
#Full job description as the only predictor

# Part B2 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from operator import itemgetter
from sklearn.metrics import classification_report
import csv
import os

In [ ]:
df_text = df_train[['FullDescription','HighSalary']]
df_text['FullDescription'] = df_text['FullDescription'].dropna().str.lower().str.replace('\d+', '')


In [ ]:
# test = vectorizer.fit(df_text['FullDescription'])
# vectorizer = TfidfVectorizer(use_idf=False,stop_words='english')

In [ ]:
#Split Train and Test sets
#Split descriptions from salary range
X_train, X_test, y_train, y_test = train_test_split(df_text['FullDescription'],
                                                    df_text['HighSalary'],
                                                    test_size=0.3, random_state=1)

In [ ]:
#Like the document term matrix, but filled with 1's and 0's instead of TFIDF scores
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.transform(X_test)

In [ ]:
#Fit model on train data
clf = BernoulliNB().fit(X_train_counts, y_train)

In [ ]:
#Predict using X_test data
prediction = clf.predict(X_test_counts)

In [ ]:
#Percent correct overall
metrics.accuracy_score(y_test, prediction) 

In [ ]:
#Row 1 is all the ones low salary
#Column 1 is all the ones predicted low salary
metrics.confusion_matrix(y_test, prediction)

In [ ]:
#Baseline accuracy = if we guessed 'No' for all of them
1-y_test.mean()